In [ ]:
# append path to find utils module in urbansim
import os 
import sys
cwd = os.getcwd() 
parentdir =  os.path.abspath(os.path.join(cwd, os.pardir))
sys.path.append(parentdir) # to get path to utils module



import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from database import get_connection_string

db_connection_string = get_connection_string('..\data\config.yml', 'mssql_db')
mssql_engine = create_engine(db_connection_string)

In [ ]:
sched_dev_parcel_sql = '''
SELECT [site_id]
      ,[parcel_id]
      ,[capacity_3]
      ,[sfu_effective_adj]
      ,[mfu_effective_adj]
      ,[mhu_effective_adj]
      ,[notes]
      ,[editor]
  FROM [urbansim].[urbansim].[scheduled_development_parcel]
  WHERE capacity_3 > 0
'''

sched_dev_df = pd.read_sql(sched_dev_parcel_sql, mssql_engine)

In [ ]:
len(sched_dev_df)

In [ ]:
priority_for_each_site = sched_dev_df.drop_duplicates(subset='site_id',keep='first').copy()
priority_for_each_site.head()

In [ ]:
len(priority_for_each_site)

In [ ]:
sites = sched_dev_df.site_id.unique()
len(sites)

In [ ]:
np.random.seed(9)
random_num = np.random.randint(2,11,size=(len(sites)))
# random_num

In [ ]:
random_num

In [ ]:
priority_for_each_site['priority'] = random_num

In [ ]:
priority_for_each_site = priority_for_each_site[['site_id','priority']].copy()

In [ ]:
ref_sql = '''
SELECT [siteid] as site_id
      ,[sitename]
      ,[source]
      ,coalesce(compdate,startdate) as date
      ,[startdate]
      ,[compdate]
  FROM [urbansim].[ref].[scheduled_development_site]
  WHERE (sfu>0 or mfu>0 or mhu>0) and status != 'completed'
  '''
ref = pd.read_sql(ref_sql,mssql_engine)

In [ ]:
len(ref)

In [ ]:
sites_w_dates = pd.merge(priority_for_each_site,ref,on='site_id',how='left')

In [ ]:
sites_w_dates['diff'] =  sites_w_dates['compdate'] - sites_w_dates['startdate']

In [ ]:
# sites_w_dates.loc[~sites_w_dates.startdate.isnull()].head()

In [ ]:
# 3*365
1035/365

In [ ]:
len(sites_w_dates.loc[sites_w_dates['diff']>'1000 days'])

In [ ]:
sites_w_dates.loc[sites_w_dates['diff']>'1000 days']

individual cases:

In [ ]:
sites_w_dates.sort_values(by='diff',ascending=False)

In [ ]:
sites_w_dates['diff'].max()

In [ ]:
print(sites_w_dates.sort_values(by='compdate'))

In [ ]:
sites_w_dates.dtypes

In [ ]:
priority_for_each_site.head()

In [ ]:
pd.DataFrame({'site_count': priority_for_each_site.
              groupby(['priority']).site_id.size()}).reset_index()

In [ ]:
sched_dev_w_priority = pd.merge(sched_dev_df,priority_for_each_site, \
                            left_on='site_id',right_on='site_id',how='left')
# sched_dev_w_year.head()
pd.DataFrame({'units_by_year': sched_dev_w_priority.
              groupby(["priority"]).capacity_3.sum()}).reset_index()

In [ ]:
sched_dev_w_priority['sched_version_id'] = 102

In [ ]:
sched_dev_for_db = sched_dev_w_priority[['sched_version_id','site_id','parcel_id','capacity_3',\
                                         'priority']].copy()

In [ ]:
len(sched_dev_for_db)

In [ ]:
sched_dev_for_db.capacity_3.sum()

In [ ]:
sched_dev_for_db.head()

In [ ]:
sched_dev_for_db.loc[sched_dev_for_db.site_id==1038]

In [ ]:
sched_dev_for_db.loc[sched_dev_for_db.site_id==1094]

In [ ]:
#sched_dev_for_db.to_csv('test.csv')

In [ ]:
# We will probably need to specify years for certain projects
#sched_dev_for_db['build_year'] = np.nan
#if site_id == xxxx:
#    sched_dev_for_db['build_year'] = yyyy